In [154]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from sklearn.preprocessing import OneHotEncoder
from sklearn.multioutput import MultiOutputClassifier
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import PoissonRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import datetime

In [155]:
df = pd.read_excel('../../../data/dataset.xlsx')
#df.columns = df.columns.str.lower()
#df = df.drop(['region','country'],axis=1)
df['WEEK'] = df['WEEK'].dt.strftime('%Y-%m-%d')
df['WEEK'] = pd.to_datetime(df['WEEK'])

In [156]:
df.head()

,WEEK,REGION,COUNTRY,ADMIN1,EVENT_TYPE,SUB_EVENT_TYPE,EVENTS,FATALITIES,POPULATION_EXPOSURE,DISORDER_TYPE,CENTROID_LATITUDE,CENTROID_LONGITUDE
0,2022-03-05,Europe,Ukraine,Chernihiv,Battles,Armed clash,11,70,219865.0,Political violence,51.3624,31.8357
1,2022-03-05,Europe,Ukraine,Chernihiv,Battles,Government regains territory,1,0,3611.0,Political violence,51.3624,31.8357
2,2022-03-05,Europe,Ukraine,Chernihiv,Explosions/Remote violence,Grenade,1,3,5383.0,Political violence,51.3624,31.8357
3,2022-03-05,Europe,Ukraine,Chernihiv,Explosions/Remote violence,Remote explosive/landmine/IED,1,3,3610.0,Political violence,51.3624,31.8357
4,2022-03-05,Europe,Ukraine,Chernihiv,Explosions/Remote violence,Shelling/artillery/missile attack,11,6,213962.0,Political violence,51.3624,31.8357


In [157]:
def statistic(name,df, columns):
    if name == 'avg':
        try:
            return np.mean(df[columns],axis=0)
        except:
            return "Provided column isn't numerical"  
    elif name == 'std':
        try:
            return np.std(df[columns],axis=0)
        except:
            return "Provided column isn't numerical"
    elif name == 'sum':
        try:
            return np.sum(df[columns],axis=0)
        except:
            return "Provided column isn't numerical"
    else:
        return 'Wrong stat'

In [158]:
df['DISORDER_TYPE']

0            Political violence
1            Political violence
2            Political violence
3            Political violence
4            Political violence
                  ...          
11250        Political violence
11251        Political violence
11252    Strategic developments
11253    Strategic developments
11254            Demonstrations
Name: DISORDER_TYPE, Length: 11255, dtype: object

In [159]:
df.columns

Index(['WEEK', 'REGION', 'COUNTRY', 'ADMIN1', 'EVENT_TYPE', 'SUB_EVENT_TYPE',
       'EVENTS', 'FATALITIES', 'POPULATION_EXPOSURE', 'DISORDER_TYPE',
       'CENTROID_LATITUDE', 'CENTROID_LONGITUDE'],
      dtype='object')

In [160]:
def statistic_over(df, columns, statistics, oblast=None, start_date=None, end_date=None, event_types=None):
    if start_date:
        df_filterd = df[(df['WEEK'] >= start_date) & (df['WEEK'] <= end_date)]
        return statistic(statistics,df_filterd,columns)
    elif oblast:
        df_region = df[df["ADMIN1"] == oblast]
        return statistic(statistics,df_region, columns)
    elif event_types:
        if type(event_types) != list():
            event_types = [event_types]
        stat = dict()
        for event in event_types:
            df_temp = df[df['EVENT_TYPE'] == event][columns]
            stat[event] = statistic(statistics, df_temp, columns)
        return stat


In [161]:
statistic_over(df, ['FATALITIES','EVENTS'], 'sum',event_types='Battles')

{'Battles': FATALITIES    156386
 EVENTS         55486
 dtype: int64}

In [162]:
def statistic_over_date(start_date, end_date, columns, statistics):
    df_filterd = df[(df['date'] >= start_date) & (df['date'] <= end_date)]
    return statistic(statistics,df_filterd,columns)

In [163]:
def statistic_over_region(oblast,columns,statistics):
    df_region = df[df["oblast'"] == oblast]
    return statistic(statistics,df_region, columns)

In [164]:
def statistics_over_event_type(df, columns, event_types,statistics):
    stat = dict()
    for event in event_types:
        df_temp = df[df['event_type'] == event][columns]
        stat[event] = statistics(statistics, df_temp, columns)
    return stat

In [165]:
def linear_plot(df,start_date, end_date, column, cumulative = False):
    x = df[(df['date'] >= start_date) & (df['date'] <= end_date)]
    if cumulative:
        y = x[column].cumsum()
    else:
        y = x[column]
    plt.figure(figsize=(15,5))
    plt.plot(x['date'], y)
    plt.title(f'{column} over time: {start_date} - {end_date}')
    plt.xlabel('Date')
    plt.ylabel(column)
    plt.show()

In [166]:
def predict_event_count(df,city=None):
    df_long = df[['WEEK','ADMIN1']].copy()
    df_long.head()
    if city:
        cities = [city]
    else:
        cities = df_long['ADMIN1'].unique()
    event_count = dict()
    for city in cities:
        model = ARIMA(df_long[df_long['ADMIN1']==city]['WEEK'].value_counts().sort_index(), order=(2, 1, 2))  # you can tune (p,d,q)
        model_fit = model.fit()
        forecast = model_fit.forecast(steps=1)
        event_count[city] = np.round(forecast.iloc[0])
    return event_count

In [167]:
def predict_coord(df,city=None):

    df_coord = df[['WEEK', 'ADMIN1', 'CENTROID_LATITUDE', 'CENTROID_LONGITUDE']].copy()

    encoder = OneHotEncoder(sparse_output=False)
    X_admin = encoder.fit_transform(df[['ADMIN1']])

    df_coord['week_num'] = df_coord['WEEK'].dt.isocalendar().week

    X = np.hstack([X_admin, df_coord[['week_num']].values])
    y = df[['CENTROID_LATITUDE', 'CENTROID_LONGITUDE']]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train) 

    y_pred = model.predict(X_test)
    
    mse = mean_squared_error(y_test, y_pred)
    next_week = pd.to_datetime(df_coord['WEEK'].iloc[-1] + datetime.timedelta(days=7)) 
    
    week_num = next_week.isocalendar()[1]
    cities = df_coord['ADMIN1'].unique()


    coord = dict()

    if city:
        cities = [city]
    else:
        cities = df_coord['ADMIN1'].unique()


    for city in cities:
        X_new = np.hstack([encoder.transform([[city]]), [[week_num]]])
        pred_coords = model.predict(X_new)
        coord[city] = pred_coords[0]

    return coord

In [168]:
def create_prediction_df(df):
    df_coord = predict_coord(df)
    df_event_count = predict_event_count(df)
    rows = []
    for city, count in df_event_count.items():
        for i in range(int(count)):
            rows.append([city])
    
    df_pred = pd.DataFrame(rows, columns=['City'])
    df_pred['WEEK_NUM'] = 39

    df_pred['CENTROID_LATITUDE'] = df_pred['City'].apply(lambda x: df_coord[x][0])
    df_pred['CENTROID_LONGITUDE'] = df_pred['City'].apply(lambda x: df_coord[x][1])

    lags = 3
    lag_values = {}

    for city, group in df.groupby('ADMIN1'):
        
        group = group.sort_values('WEEK')


        last_vals = group['EVENTS'].tail(lags).tolist()

        lag_values[city] = last_vals

    for col in ['EVENTS', 'FATALITIES']:
        for i in range(1, lags+1):
            df_pred[f'{col}_lag{i}'] = df_pred['City'].apply(lambda city: lag_values[city][-i])


    for col in ['EVENTS', 'FATALITIES']:
        df_pred[f'{col}_rolling_mean'] = (
            df.sort_values(['ADMIN1', 'WEEK'])         # ensure correct order
              .groupby('ADMIN1')[col]                 # group by city/admin
              .transform(lambda x: x.shift(1)        # avoid including current week
                                 .rolling(3, min_periods=1)
                                 .mean())
        )

    rolling_cols = ['EVENTS_rolling_mean', 'FATALITIES_rolling_mean']
    df_pred[rolling_cols] = df_pred.groupby('City')[rolling_cols].transform(lambda x: x.bfill().ffill())

   
    for col in rolling_cols:
        df_pred.iloc[-1, df_pred.columns.get_loc(col)] = df_pred.iloc[-2][col]

    #events = ['Battles', 'Explosions/Remote violence', 'Protests', 'Riots', 'Strategic developments', 'Violence against civilians']

    #df_pred['EVENT_TYPE'] = np.random.choice(events, size=len(df_pred))

    df_pred = pd.get_dummies(df_pred, columns=['City'], prefix='ADMIN1')
    
    #df_pred = pd.get_dummies(df_pred, columns=['EVENT_TYPE'], prefix='EVENT_TYPE')
    df_pred = df_pred[sorted(df_pred.columns)]

    return df_pred

In [169]:
create_prediction_df(df)

c:\Users\kumak\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
c:\Users\kumak\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
c:\Users\kumak\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
c:\Users\kumak\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
c:\Users\kumak\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2749: UserW

,ADMIN1_Cape Fiolent,ADMIN1_Cherkasy,ADMIN1_Chernihiv,ADMIN1_Chernivtsi,ADMIN1_Coast of Sevastopol,ADMIN1_Crimea,ADMIN1_Dnipropetrovsk,ADMIN1_Donetsk,ADMIN1_Eastern Black Sea,ADMIN1_Ivano-Frankivsk,...,CENTROID_LONGITUDE,EVENTS_lag1,EVENTS_lag2,EVENTS_lag3,EVENTS_rolling_mean,FATALITIES_lag1,FATALITIES_lag2,FATALITIES_lag3,FATALITIES_rolling_mean,WEEK_NUM
0,False,False,True,False,False,False,False,False,False,False,...,31.835700,2,49,26,11.000000,2,49,26,70.000000,39
1,False,False,True,False,False,False,False,False,False,False,...,31.835700,2,49,26,11.000000,2,49,26,70.000000,39
2,False,False,True,False,False,False,False,False,False,False,...,31.835700,2,49,26,6.000000,2,49,26,35.000000,39
3,False,False,False,False,False,True,False,False,False,False,...,34.142300,5,1,7,4.333333,5,1,7,24.333333,39
4,False,False,False,False,False,True,False,False,False,False,...,34.142300,5,1,7,1.000000,5,1,7,2.000000,39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,False,False,False,False,False,False,False,False,False,False,...,30.114900,2,1,2,5.000000,2,1,2,40.000000,39
69,False,False,False,False,False,False,False,False,False,False,...,23.035000,1,2,2,28.500000,1,2,2,58.000000,39
70,False,False,False,True,False,False,False,False,False,False,...,25.774500,1,1,1,19.333333,1,1,1,38.666667,39
71,False,False,False,False,True,False,False,False,False,False,...,33.466313,1,1,1,18.666667,1,1,1,25.333333,39


In [170]:
def predict_event_type(df,unencoded=False):
    df_event = df[['WEEK', 'ADMIN1', 'EVENT_TYPE', 'EVENTS', 'FATALITIES', 'CENTROID_LATITUDE', 'CENTROID_LONGITUDE']].copy()
   
    lags = 3 
    for col in ['EVENTS','FATALITIES']:
        for lag in range(1, lags+1):
            df_event[f'{col}_lag{lag}'] = df_event.groupby('ADMIN1')[col].shift(lag)


    lag_cols = [f'{col}_lag{lag}' for col in ['EVENTS','FATALITIES'] for lag in range(1, lags+1)]

    window = 3 


    for col in ['EVENTS', 'FATALITIES']:
        df_event[f'{col}_rolling_mean'] = df_event.groupby('ADMIN1')[col].transform(
            lambda x: x.shift(1).rolling(window, min_periods=1).mean()
        )

    df_event[lag_cols] = df_event.groupby('ADMIN1')[lag_cols].transform(lambda x: x.bfill())

    cols_to_fill = [col for col in df_event.columns if 'lag' in col or 'rolling' in col]


    df_event[cols_to_fill] = df_event.groupby('ADMIN1')[cols_to_fill].transform(lambda x: x.bfill().ffill())


    df_event = df_event.drop(['EVENTS','FATALITIES'],axis=1)


    df_event = pd.get_dummies(df_event, columns=['ADMIN1'], prefix='ADMIN1')
    df_event = pd.get_dummies(df_event, columns=['EVENT_TYPE'], prefix='EVENT_TYPE')
    df_event['WEEK_NUM'] = df_event['WEEK'].dt.isocalendar().week
    df_event = df_event.drop('WEEK',axis=1)

    admin_cols = [col for col in df_event.columns if col.startswith('ADMIN1_')]
    event_type_cols = [col for col in df_event.columns if col.startswith('EVENT_TYPE_')]
    feature_cols = ['WEEK_NUM', 'CENTROID_LATITUDE', 'CENTROID_LONGITUDE','EVENTS_rolling_mean', 'FATALITIES_rolling_mean'] + [f'{col}_lag{lag}' for col in ['EVENTS','FATALITIES'] for lag in range(1, lags+1)] + admin_cols


    
    X = df_event[sorted(feature_cols)]
    y = df_event[event_type_cols]


    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)


    model = MultiOutputClassifier(RandomForestClassifier(n_estimators=200, class_weight='balanced', random_state=42))

    model.fit(X_train, y_train)


    X_new = create_prediction_df(df)

    y_pred = model.predict(X_new)
    event_types = ['Battles', 'Explosions/Remote violence', 'Protests', 'Riots', 'Strategic developments', 'Violence against civilians']

    if unencoded:
        y_pred_combined = []
        for i in range(len(y_pred)):   # use y_pred length
            for j, col in enumerate(event_types):
                if y_pred[i][j]:   # True for this class
                    y_pred_combined.append(event_types[j])
                    break
        return y_pred_combined
    else:
        return pd.DataFrame(y_pred, columns=event_types)
    

In [171]:
def prediction_df(df):
    x = create_prediction_df(df)
    y = predict_event_type(df)

    return pd.concat([x,y], axis=1)

In [172]:
#predict_event_type(df)

In [173]:
def predict_fatalities(df, X_new):
    df_fatal = df[['ADMIN1','WEEK', 'CENTROID_LATITUDE', 'CENTROID_LONGITUDE', 'EVENTS', 'FATALITIES','EVENT_TYPE']].copy()
    lags = 3 
    for col in ['EVENTS','FATALITIES']:
        for lag in range(1, lags+1):
            df_fatal[f'{col}_lag{lag}'] = df_fatal.groupby('ADMIN1')[col].shift(lag)
    lag_cols = [f'{col}_lag{lag}' for col in ['EVENTS','FATALITIES'] for lag in range(1, lags+1)]
    window = 3 
    for col in ['EVENTS', 'FATALITIES']:
        df_fatal[f'{col}_rolling_mean'] = df_fatal.groupby('ADMIN1')[col].transform(
            lambda x: x.shift(1).rolling(window, min_periods=1).mean()
        )
    df_fatal[lag_cols] = df_fatal.groupby('ADMIN1')[lag_cols].transform(lambda x: x.bfill())
    cols_to_fill = [col for col in df_fatal.columns if 'lag' in col or 'rolling' in col]
    df_fatal[cols_to_fill] = df_fatal.groupby('ADMIN1')[cols_to_fill].transform(lambda x: x.bfill().ffill())

    df_fatal = pd.get_dummies(df_fatal, columns=['ADMIN1'], prefix='ADMIN1')
    df_fatal = pd.get_dummies(df_fatal, columns=['EVENT_TYPE'], prefix='EVENT_TYPE')

    df_fatal['WEEK_NUM'] = df_fatal['WEEK'].dt.isocalendar().week
    df_fatal = df_fatal.drop('WEEK',axis=1)

    admin_cols = [col for col in df_fatal.columns if col.startswith('ADMIN1_')]
    event_cols = [col for col in df_fatal.columns if col.startswith('EVENT_TYPE_')]
    feature_cols = ['WEEK_NUM', 'CENTROID_LATITUDE', 'CENTROID_LONGITUDE','EVENTS_rolling_mean', 'FATALITIES_rolling_mean'] + [f'{col}_lag{lag}' for col in ['EVENTS','FATALITIES'] for lag in range(1, lags+1)] + admin_cols + event_cols

    X = df_fatal[sorted(feature_cols)]
    y_events = df_fatal['EVENTS']
    y_fatalities = df_fatal['FATALITIES']

    X_train, X_test, y_train_events, y_test_events = train_test_split(X, y_events, test_size=0.2, shuffle=False)
    _, _, y_train_fatal, y_test_fatal = train_test_split(X, y_fatalities, test_size=0.2, shuffle=False)

    params = {
        'objective': 'poisson',   
        'metric': 'rmse',         
        'learning_rate': 0.05,
        'num_leaves': 31,
        'min_data_in_leaf': 20,
        'verbosity': -1,
        'seed': 42
    }

    events_model = LGBMRegressor(
        objective='poisson',
        n_estimators=1000,
        learning_rate=0.05,
        num_leaves=31,
        min_data_in_leaf=20,
        random_state=42
    )

    events_model.fit(
        X_train, y_train_events,
        eval_set=[(X_test, y_test_events)],
        eval_metric='rmse',

    )

    # FATALITIES model
    fatal_model = LGBMRegressor(
        objective='poisson',
        n_estimators=1000,
        learning_rate=0.05,
        num_leaves=31,
        min_data_in_leaf=20,
        random_state=42
    )

    fatal_model.fit(
        X_train, y_train_fatal,
        eval_set=[(X_test, y_test_fatal)],
        eval_metric='rmse',

    )

    y_pred_events = events_model.predict(X_new)
    y_pred_fatal = fatal_model.predict(X_new)

    # Clip predictions to avoid negative counts
    y_pred_events = pd.DataFrame(np.round(np.clip(y_pred_events, 0, None)),columns=['Events'])
    y_pred_fatal = pd.DataFrame(np.round(np.clip(y_pred_fatal, 0, None)), columns=['Fatalities'])

    prediction = pd.concat([y_pred_events, y_pred_fatal],axis=1)
    return prediction

In [174]:
def predict(df):
    x = prediction_df(df)

    y = predict_fatalities(df, x)

    pred = pd.concat([x, y], axis=1)

    event_cols = ['Battles', 'Explosions/Remote violence', 'Protests', 
              'Riots', 'Strategic developments', 'Violence against civilians']

    pred['EVENT_TYPE'] = pred[event_cols].idxmax(axis=1)
    
    admin_cols = [col for col in pred.columns if col.startswith('ADMIN1_')]
    
    pred['ADMIN1'] = pred[admin_cols].idxmax(axis=1).str.replace('ADMIN1_', '')
    
    cols_to_drop = event_cols + admin_cols +['EVENTS_lag1', 'EVENTS_lag2', 'EVENTS_lag3', 'EVENTS_rolling_mean', 'FATALITIES_lag1', 'FATALITIES_lag2', 'FATALITIES_lag3', 'FATALITIES_rolling_mean']

    year = 2025
    week = 39
    week_end = datetime.date.fromisocalendar(year, week, 7)
    pred = pred.drop(columns=cols_to_drop)
    pred['WEEK_NUM'] = week_end

    new_order=['WEEK_NUM', 'ADMIN1','EVENT_TYPE','Events','Fatalities','CENTROID_LATITUDE','CENTROID_LONGITUDE']

    pred = pred[new_order]
    
    return pred

In [175]:
pred = predict(df)

c:\Users\kumak\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
c:\Users\kumak\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
c:\Users\kumak\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
c:\Users\kumak\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
c:\Users\kumak\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2749: UserW

[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000510 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2171
[LightGBM] [Info] Number of data points in the train set: 9004, number of used features: 42
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Start training from score 2.872203
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] 

In [177]:
pred

,WEEK_NUM,ADMIN1,EVENT_TYPE,Events,Fatalities,CENTROID_LATITUDE,CENTROID_LONGITUDE
0,2025-09-28,Chernihiv,Explosions/Remote violence,3.0,0.0,51.362400,31.835700
1,2025-09-28,Chernihiv,Explosions/Remote violence,3.0,0.0,51.362400,31.835700
2,2025-09-28,Chernihiv,Explosions/Remote violence,3.0,0.0,51.362400,31.835700
3,2025-09-28,Crimea,Explosions/Remote violence,2.0,1.0,45.306000,34.142300
4,2025-09-28,Crimea,Explosions/Remote violence,2.0,2.0,45.306000,34.142300
...,...,...,...,...,...,...,...
68,2025-09-28,Western Black Sea,Explosions/Remote violence,2.0,0.0,43.978300,30.114900
69,2025-09-28,Zakarpattia,Battles,2.0,0.0,48.496900,23.035000
70,2025-09-28,Chernivtsi,Battles,3.0,0.0,48.200200,25.774500
71,2025-09-28,Coast of Sevastopol,Explosions/Remote violence,4.0,0.0,44.620956,33.466313


In [176]:
pred

,WEEK_NUM,ADMIN1,EVENT_TYPE,Events,Fatalities,CENTROID_LATITUDE,CENTROID_LONGITUDE
0,2025-09-28,Chernihiv,Explosions/Remote violence,3.0,0.0,51.362400,31.835700
1,2025-09-28,Chernihiv,Explosions/Remote violence,3.0,0.0,51.362400,31.835700
2,2025-09-28,Chernihiv,Explosions/Remote violence,3.0,0.0,51.362400,31.835700
3,2025-09-28,Crimea,Explosions/Remote violence,2.0,1.0,45.306000,34.142300
4,2025-09-28,Crimea,Explosions/Remote violence,2.0,2.0,45.306000,34.142300
...,...,...,...,...,...,...,...
68,2025-09-28,Western Black Sea,Explosions/Remote violence,2.0,0.0,43.978300,30.114900
69,2025-09-28,Zakarpattia,Battles,2.0,0.0,48.496900,23.035000
70,2025-09-28,Chernivtsi,Battles,3.0,0.0,48.200200,25.774500
71,2025-09-28,Coast of Sevastopol,Explosions/Remote violence,4.0,0.0,44.620956,33.466313
